In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
pd.set_option('display.max_rows',1000)
from datetime import datetime
from kmodes.kprototypes import KPrototypes
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
plt.style.use('seaborn')
import seaborn as sns
matplotlib.rcParams['font.family']='Malgun Gothic'
matplotlib.rcParams['axes.unicode_minus'] = False
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

from sklearn.cluster import KMeans

# Import module for data visualization
from plotnine import *
import plotnine
# Import module for k-protoype cluster

import warnings
warnings.filterwarnings('ignore', category = FutureWarning)
# Format scientific notation from Pandas
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# 데이터 불러오기

In [2]:
# 군집 이동 데이터 불러오기
df = pd.read_csv('data/cluster/clust_1st_7_비인도데프.csv')
df

,id,2017_1Q,2017_2Q,2017_3Q,2017_4Q,2018_1Q,2018_2Q,2018_3Q,2018_4Q,2019_1Q,2019_2Q,2019_3Q,2019_4Q,2020_1Q,2020_2Q,2020_3Q,2020_4Q,2021_1Q,2021_2Q
0,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2
1,경기도 고양덕양구 고양동_고양동우방유쉘아파트_85,2,2,2,2,2,2,2,1,2,2,2,2,2,2,2,2,2,2
2,경기도 고양덕양구 고양동_삼성_60,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,경기도 고양덕양구 고양동_삼성_85,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2
4,경기도 고양덕양구 고양동_윤창_59,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21122,인천광역시 중구 항동7가_라이프비취2차_72,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
21123,인천광역시 중구 항동7가_라이프비취3차_126,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
21124,인천광역시 중구 항동7가_라이프비취3차_164,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,2,2
21125,인천광역시 중구 항동7가_라이프비취3차_85,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [3]:
# id 따로 빼기
df_id = df.id

# 데이터프레임에서 id 제거
df = df.drop('id', axis=1)
df

,2017_1Q,2017_2Q,2017_3Q,2017_4Q,2018_1Q,2018_2Q,2018_3Q,2018_4Q,2019_1Q,2019_2Q,2019_3Q,2019_4Q,2020_1Q,2020_2Q,2020_3Q,2020_4Q,2021_1Q,2021_2Q
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2
1,2,2,2,2,2,2,2,1,2,2,2,2,2,2,2,2,2,2
2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2
4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21122,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
21123,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
21124,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,2,2
21125,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


# 패턴 파악하기

In [4]:
from collections import deque

# 패턴 컬럼 추가
df['pattern'] ='nan'
for i in range(df.shape[0]):
    sample = deque(df.iloc[i].values)
    sample = deque(map(str, sample))
    sample.pop()
    pocket = []
    pocket.append(sample.popleft())
    while sample:
        sp = sample.popleft()
        if sp == pocket[-1]:
            continue
        else:
            pocket.append(sp)
    p = '-'.join(pocket)
    df.loc[i, 'pattern'] = p
df 

,2017_1Q,2017_2Q,2017_3Q,2017_4Q,2018_1Q,2018_2Q,2018_3Q,2018_4Q,2019_1Q,2019_2Q,2019_3Q,2019_4Q,2020_1Q,2020_2Q,2020_3Q,2020_4Q,2021_1Q,2021_2Q,pattern
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1-2
1,2,2,2,2,2,2,2,1,2,2,2,2,2,2,2,2,2,2,2-1-2
2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1-2
4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21122,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
21123,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
21124,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,2,2,1-2
21125,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [5]:
# pattern 컬럼 추가
df= pd.concat([df_id, df], axis=1)
df

,id,2017_1Q,2017_2Q,2017_3Q,2017_4Q,2018_1Q,2018_2Q,2018_3Q,2018_4Q,2019_1Q,2019_2Q,2019_3Q,2019_4Q,2020_1Q,2020_2Q,2020_3Q,2020_4Q,2021_1Q,2021_2Q,pattern
0,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1-2
1,경기도 고양덕양구 고양동_고양동우방유쉘아파트_85,2,2,2,2,2,2,2,1,2,2,2,2,2,2,2,2,2,2,2-1-2
2,경기도 고양덕양구 고양동_삼성_60,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,경기도 고양덕양구 고양동_삼성_85,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1-2
4,경기도 고양덕양구 고양동_윤창_59,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21122,인천광역시 중구 항동7가_라이프비취2차_72,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
21123,인천광역시 중구 항동7가_라이프비취3차_126,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
21124,인천광역시 중구 항동7가_라이프비취3차_164,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,2,2,1-2
21125,인천광역시 중구 항동7가_라이프비취3차_85,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1



---


# 패턴 검색

In [6]:
# 2차 군집 포함한 마스터테이블 불러오기
df2 = pd.read_csv('data/cluster/2차군집8_비인도데프.csv')

In [7]:
# 패턴 검색 함수
def go(df):
    vc =df.pattern.value_counts()
    print('패턴 수: ', vc.shape)
    print(vc)
    key1 = input('검색하고 싶은 패턴')
    print(df[df.pattern == key1].id.tolist())
    key2 = input('검색할 id')
    display(df2[df2.id==key2])
    

In [18]:
# 함수 실행
go(df)

패턴 수:  (378,)
1                              3388
2-4                            3287
4-5                            2595
1-2                            1406
5-6                            1149
2                              1104
2-4-5                           713
4-5-6                           563
2-1-2                           451
4-5-4-5                         430
2-4-2-4                         422
6-7                             384
1-2-1-2                         332
4                               271
3-2-4                           239
5-6-5-6                         226
2-1-2-1-2                       212
2-4-2-4-5                       192
3-4-5                           171
4-2-4-5                         157
1-3                             125
2-1-2-4                          98
7                                98
3-5-6                            96
1-2-1-2-1-2                      95
4-5-4-5-6                        89
5-6-7                            87
2-3-2-4       

검색하고 싶은 패턴2-1-2
['경기도 고양덕양구 고양동_고양동우방유쉘아파트_85', '경기도 고양덕양구 고양동_푸른마을6단지_85', '경기도 고양덕양구 관산동_더퍼스트프라임_85', '경기도 고양덕양구 관산동_유승(126)_116', '경기도 고양덕양구 토당동_삼윤_85', '경기도 고양덕양구 토당동_태웅_77', '경기도 고양덕양구 행신동_샘터마을2_60', '경기도 고양덕양구 행신동_햇빛주공23_60', '경기도 고양덕양구 화정동_달빛(라이프)2_60', '경기도 고양덕양구 화정동_달빛마을(부영)2_60', '경기도 고양일산동구 마두동_강촌마을(라이프)_48', '경기도 고양일산동구 마두동_백마마을(쌍용)_51', '경기도 고양일산동구 마두동_백마마을(한성)_51', '경기도 고양일산동구 백석동_백송마을(대우)_59', '경기도 고양일산동구 백석동_백송마을(두산)_60', '경기도 고양일산동구 백석동_백송마을(삼호)_60', '경기도 고양일산동구 백석동_백송마을(우성)_53', '경기도 고양일산동구 장항동_호수마을(대우)_55', '경기도 고양일산동구 중산동_중산마을10(경남)_74', '경기도 고양일산동구 중산동_중산마을10(경남)_85', '경기도 고양일산동구 중산동_중산마을10(동신)_74', '경기도 고양일산동구 중산동_중산마을6(태영)_73', '경기도 고양일산동구 중산동_중산마을7(코오롱)_102', '경기도 고양일산동구 풍동_성원타운3_60', '경기도 고양일산동구 풍동_숲속마을8단지_52', '경기도 고양일산동구 풍동_일산풍동5차성원상떼빌_60', '경기도 고양일산서구 가좌동_가좌마을1단지양우_85', '경기도 고양일산서구 가좌동_가좌마을2단지대우_74', '경기도 고양일산서구 가좌동_청원벽산블루밍3단지_85', '경기도 고양일산서구 대화동_양우파크타운_60', '경기도 고양일산서구 대화동_장성마을(동부)_53', '경기도 고양일산서구 일산동_에이스10차_106', '경기도 고양일산서구 일산동_에이스10차_121', '경기도 고양일산서구 

검색할 id경기도 수원영통구 영통동_황골마을주공1_60


,id,quarter,전용면적_미제,거래금액,건축년도,보증금,매매상승률,전세상승률,전세가율,평당매매가,매매빈도,전세빈도,위도,경도,서울구분,6억초과,규제,평당거주비용,cluster,2nd_cluster
88866,경기도 수원영통구 영통동_황골마을주공1_60,2017_1Q,60.000,248.122,1997.000,213.158,0.990,0.914,85.909,13.785,41.000,49.000,37.263,127.080,0,0,해당 없음,0.439,2,3
88867,경기도 수원영통구 영통동_황골마을주공1_60,2017_2Q,60.000,251.543,1997.000,212.903,1.379,-0.119,84.639,13.975,47.000,35.000,37.263,127.080,0,0,해당 없음,0.451,2,3
88868,경기도 수원영통구 영통동_황골마을주공1_60,2017_3Q,60.000,249.545,1997.000,200.129,-0.794,-6.000,80.197,13.864,44.000,41.000,37.263,127.080,0,0,해당 없음,0.453,2,3
88869,경기도 수원영통구 영통동_황골마을주공1_60,2017_4Q,60.000,243.567,1997.000,191.250,-2.396,-4.437,78.521,13.531,30.000,29.000,37.263,127.080,0,0,해당 없음,0.457,2,3
88870,경기도 수원영통구 영통동_황골마을주공1_60,2018_1Q,60.000,236.372,1997.000,186.795,-2.954,-2.329,79.026,13.132,43.000,49.000,37.263,127.080,0,0,해당 없음,0.454,1,3
88871,경기도 수원영통구 영통동_황골마을주공1_60,2018_2Q,60.000,235.975,1997.000,193.000,-0.168,3.322,81.788,13.110,20.000,36.000,37.263,127.080,0,0,해당 없음,0.455,1,3
88872,경기도 수원영통구 영통동_황골마을주공1_60,2018_3Q,60.000,233.223,1997.000,191.581,-1.166,-0.735,82.145,12.957,47.000,35.000,37.263,127.080,0,0,해당 없음,0.436,1,3
88873,경기도 수원영통구 영통동_황골마을주공1_60,2018_4Q,60.000,239.857,1997.000,185.614,2.844,-3.115,77.385,13.325,63.000,54.000,37.263,127.080,0,0,해당 없음,0.434,1,3
88874,경기도 수원영통구 영통동_황골마을주공1_60,2019_1Q,60.000,229.086,1997.000,178.367,-4.491,-3.904,77.860,12.727,29.000,64.000,37.263,127.080,0,0,해당 없음,0.392,1,3
88875,경기도 수원영통구 영통동_황골마을주공1_60,2019_2Q,60.000,221.955,1997.000,159.654,-3.113,-10.492,71.931,12.331,22.000,67.000,37.263,127.080,0,0,해당 없음,0.356,1,3


### 각 2차군집의 패턴 파악해보기

In [8]:
# id, 2차군집 결과 
df_id_2nd = df2[['id','2nd_cluster']]
df_id_2nd.drop_duplicates(inplace=True)
df_id_2nd

<ipython-input-8-d406f2d2b709>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,id,2nd_cluster
0,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,1
18,경기도 고양덕양구 고양동_고양동우방유쉘아파트_85,6
36,경기도 고양덕양구 고양동_삼성_60,1
54,경기도 고양덕양구 고양동_삼성_85,1
72,경기도 고양덕양구 고양동_윤창_59,1
...,...,...
380196,인천광역시 중구 항동7가_라이프비취2차_72,1
380214,인천광역시 중구 항동7가_라이프비취3차_126,1
380232,인천광역시 중구 항동7가_라이프비취3차_164,3
380250,인천광역시 중구 항동7가_라이프비취3차_85,1


In [9]:
# 데이터프레임에 2차군집 컬럼 생성
df_merge2nd = pd.merge(df, df_id_2nd, how='left', on ='id')

# 열 재배열
df_merge2nd = df_merge2nd[['id','2nd_cluster','pattern', '2017_1Q', '2017_2Q', '2017_3Q', '2017_4Q', '2018_1Q', '2018_2Q',
       '2018_3Q', '2018_4Q', '2019_1Q', '2019_2Q', '2019_3Q', '2019_4Q',
       '2020_1Q', '2020_2Q', '2020_3Q', '2020_4Q', '2021_1Q', '2021_2Q']]
df_merge2nd

,id,2017_1Q,2017_2Q,2017_3Q,2017_4Q,2018_1Q,2018_2Q,2018_3Q,2018_4Q,2019_1Q,...,2019_3Q,2019_4Q,2020_1Q,2020_2Q,2020_3Q,2020_4Q,2021_1Q,2021_2Q,pattern,2nd_cluster
0,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,2,1-2,1
1,경기도 고양덕양구 고양동_고양동우방유쉘아파트_85,2,2,2,2,2,2,2,1,2,...,2,2,2,2,2,2,2,2,2-1-2,6
2,경기도 고양덕양구 고양동_삼성_60,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,경기도 고양덕양구 고양동_삼성_85,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,2,1-2,1
4,경기도 고양덕양구 고양동_윤창_59,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21122,인천광역시 중구 항동7가_라이프비취2차_72,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
21123,인천광역시 중구 항동7가_라이프비취3차_126,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
21124,인천광역시 중구 항동7가_라이프비취3차_164,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,2,2,2,1-2,3
21125,인천광역시 중구 항동7가_라이프비취3차_85,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


# 데이터 저장

In [18]:
df_merge2nd.to_csv('data/etc/군집이동_2차군집_패턴.csv', index=False, encoding='utf-8-sig')

---

# 2차군집 별로 패턴 분석

In [20]:
for i in range(8):
    globals()['cluster2_{}'.format(i)] = df_merge2nd[df_merge2nd['2nd_cluster'] == i]

In [188]:
go(df)

패턴 수:  (378,)
1                              3388
2-4                            3287
4-5                            2595
1-2                            1406
5-6                            1149
2                              1104
2-4-5                           713
4-5-6                           563
2-1-2                           451
4-5-4-5                         430
2-4-2-4                         422
6-7                             384
1-2-1-2                         332
4                               271
3-2-4                           239
5-6-5-6                         226
2-1-2-1-2                       212
2-4-2-4-5                       192
3-4-5                           171
4-2-4-5                         157
1-3                             125
2-1-2-4                          98
7                                98
3-5-6                            96
1-2-1-2-1-2                      95
4-5-4-5-6                        89
5-6-7                            87
2-3-2-4       

KeyboardInterrupt: Interrupted by user

In [21]:
go(cluster2_3)

패턴 수:  (46,)
1-2                    519
1-2-1-2                163
2-1-2                  101
2-1-2-1-2               59
1-2-1-2-1-2             50
1-2-3                   35
2-1-2-1-2-1-2           21
1-2-1-2-1-2-1-2         19
1-2-4                   10
1-2-3-2                  7
1-3-2                    6
1-3-2-3                  6
3-1-2                    6
1-2-1-2-3                5
1-2-3-2-3                4
3-2                      4
1-2-1-2-1-2-1-2-1-2      4
1-3-1-3-2                3
1-2-1-2-4                3
1-2-1                    3
3-1-3-1-2                3
2-1-2-1-2-1-2-1-2        3
1-3-1-3-2-4              3
2-1-2-1-2-4              2
1-2-1-2-1                2
3-1-2-1-2                2
3-2-1-2                  1
3-1-3-2-4                1
2-1-2-1                  1
1-3-2-3-2-3              1
3-2-4                    1
1-2-1-2-1-2-4            1
1-2-1-2-1-3-2            1
2-1-2-4                  1
3-1-3-2                  1
2-1-2-1-2-3-2            1
1-3-1-3-1-2    

KeyboardInterrupt: Interrupted by user

In [196]:
df_merge2nd['2nd_cluster'].value_counts()

1    4459
7    4132
0    3938
2    3078
6    2450
5    1578
3    1063
4     429
Name: 2nd_cluster, dtype: int64


---

# 동별 랭킹 구하기

In [154]:
# 군집이동 데이터 가져오기
df = pd.read_csv('data/cluster/clust_1st_7_비인도데프.csv')
df

,id,2017_1Q,2017_2Q,2017_3Q,2017_4Q,2018_1Q,2018_2Q,2018_3Q,2018_4Q,2019_1Q,2019_2Q,2019_3Q,2019_4Q,2020_1Q,2020_2Q,2020_3Q,2020_4Q,2021_1Q,2021_2Q
0,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2
1,경기도 고양덕양구 고양동_고양동우방유쉘아파트_85,2,2,2,2,2,2,2,1,2,2,2,2,2,2,2,2,2,2
2,경기도 고양덕양구 고양동_삼성_60,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,경기도 고양덕양구 고양동_삼성_85,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2
4,경기도 고양덕양구 고양동_윤창_59,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21122,인천광역시 중구 항동7가_라이프비취2차_72,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
21123,인천광역시 중구 항동7가_라이프비취3차_126,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
21124,인천광역시 중구 항동7가_라이프비취3차_164,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,2,2
21125,인천광역시 중구 항동7가_라이프비취3차_85,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [156]:
# 분기 리스트
list_quarter = ['2017_1Q','2017_2Q','2017_3Q','2017_4Q','2018_1Q','2018_2Q','2018_3Q','2018_4Q',
                '2019_1Q','2019_2Q','2019_3Q','2019_4Q','2020_1Q','2020_2Q','2020_3Q','2020_4Q','2021_1Q','2021_2Q']

# 동별 카운트
for j in range(1,8):
    for i in list_quarter:
        dong = df[df[i]==j].id.str.split('_').str[0]
        cols = dong.value_counts().index
        values = dong.value_counts()
        
        globals()['c{}_{}'.format(j,i)] = pd.DataFrame([cols, values])

In [157]:
# 랭크 만들기 함수
def makeRank(n):
    dong_rank_171q = globals()['c{}_2017_1Q'.format(n)].values[0].tolist()
    dong_rank_212q = globals()['c{}_2021_2Q'.format(n)].values[0].tolist()
    status = []
    for dong in dong_rank_212q:
        try:
            idx_171q = dong_rank_171q.index(dong)
        except:
#             print('새로등재')
            status.append('새로등재')
            continue

        idx_212q = dong_rank_212q.index(dong)
    #     print(idx_171q,idx_212q)

        if idx_171q<idx_212q:
#             print(dong + ' 순위 내려감')
            status.append('내려감')

        elif idx_171q==idx_212q:
#             print(dong + '순위 그대로')
            status.append('그대로')
        else:
#             print(dong + '순위 올라감')
            status.append('올라감')


    correct_212q = pd.concat([ globals()['c{}_2021_2Q'.format(n)], pd.DataFrame(status).T])
    # display(c0_2017_1Q)
    # display(correct_212q)
    correct_212q = correct_212q.T
    correct_212q.columns = ['Dong', 'Count', 'Rank']
    return correct_212q

In [158]:
# 랭킹 테이블 만들어 저장
for i in range(1,8):
    globals()['c{}'.format(i)] = makeRank(i)
    globals()['c{}'.format(i)].to_csv('랭킹_'+str(i)+'_비인도데프.csv', index=False, encoding='utf-8-sig')


---


# 전세가율 테이블 생성

In [74]:
# 데이터 불러오기
df = pd.read_csv('data/cluster/2차군집8_비인도데프.csv')
print(df.shape)


df17 = df[df.quarter=="2017_1Q"]    # 17년도 1분기
df21 = df[df.quarter=="2021_2Q"]    # 21년도 2분기

(380286, 20)

In [77]:
# 보증금 Series
jeonse_17 = df17.보증금.reset_index(drop=True)    # 17년도 1분기
jeonse_21 = df21.보증금.reset_index(drop=True)    # 21년도 2분기

In [81]:
# 17년도 1분기 데이터에 전세 전체상승률 생성
df17['전세전체상승률'] = ((jeonse_21 - jeonse_17) / jeonse_17) * 100

# 17년도 1분기 데이터에 매매 전체상승률 컬럼 생성
df17['매매전체상승률'] = (((ccc - aaa) / aaa) * 100)

<ipython-input-81-7b1237015206>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [201]:
# 전세가율 Series
rate_deposit_17 = df17.전세가율.reset_index(drop=True)    # 17년도 1분기
rate_deposit_21 = df21.전세가율.reset_index(drop=True)    # 21년도 2분기

# 데이터 프레임화
rate_deposit_17 = pd.DataFrame(rate_deposit_17)
rate_deposit_21 = pd.DataFrame(rate_deposit_21)

In [202]:
# 전세가율 테이블 생성
df_result = pd.merge(rate_deposit_17, rate_deposit_21, how="left", left_index=True, right_index=True)
df_result.columns = ['전세가율_17', '전세가율_21']
df_result

,전세가율_17,전세가율_21
0,83.542,68.998
1,82.131,69.474
2,77.348,62.143
3,82.759,105.526
4,88.339,93.333
...,...,...
21122,79.167,67.416
21123,52.910,45.045
21124,41.379,62.500
21125,60.708,72.301


In [204]:
# id 컬럼 추출
col_id = pd.DataFrame(df17.id.unique())

# 전세가율 테이블에 id 컬럼 추가
df_result['id'] = col_id[0]
df_result.head()

,전세가율_17,전세가율_21,id
0,83.542,68.998,경기도 고양덕양구 고양동_고양동우방유쉘아파트_60
1,82.131,69.474,경기도 고양덕양구 고양동_고양동우방유쉘아파트_85
2,77.348,62.143,경기도 고양덕양구 고양동_삼성_60
3,82.759,105.526,경기도 고양덕양구 고양동_삼성_85
4,88.339,93.333,경기도 고양덕양구 고양동_윤창_59


In [207]:
# 조건: 17년도 전세가율이 21년도 전세가율이 더 클 때 
condition = (df_result.전세가율_17 < df_result.전세가율_21)

# 17년도 전세가율 < 21년도 전세가율 조건 컬럼 추가
df_result['17 < 21'] = condition

# 위도 경도 추가
df_result['위도'] = df17.위도
df_result['경도'] = df17.경도

# 2차군집 컬럼
df_result['2nd_cluster'] = df_result['2nd_cluster']

# 전체상승률 컬럼 추가
df_result['매매전체상승률'] = df17.매매전체상승률    # 매매상승률
df_result['전세전체상승률'] = df17.전세전체상승률    # 전세상승률

df_result

# 데이터 저장

In [130]:
df_result.to_csv("data/etc/전세가율 Table_비인도데프.csv", index=False, encoding="utf-8-sig")